
# QCI
#### Author: Jan T. (IR) en Team D&A

In [1]:
###Load packages
import pandas       as pd   #required for data wrangling
import pingouin     as pg   #required for Cronbach's Alpha
import numpy        as np   #required for NaN

In [4]:
### Load data
qcidata = pd.read_csv("QCIdata_volledig_anoniem.csv", sep=";")

## Data cleaning
Remove all values 0, 6, 7 and 8 (corresponding to some missing data type), since they will influence the outcome. In this case, the best method is to replace these values by 'NaN' (not a number). 

In [5]:
missing_numbers = [0,6,7,8]
qcidata_clean = qcidata.replace(missing_numbers[0], np.NaN)

## Interne consistentie
Om interne consistentie per subschaal te meten is [Cronbach's Alpha](https://www.scribbr.nl/statistiek/cronbachs-alpha/) gebruikt, deze is geïnterpreteerd met gebruik van de volgende [bron](https://towardsdatascience.com/cronbachs-alpha-theory-and-application-in-python-d2915dd63586).

### Definieer subschalen
Elke subschaal bestaat uit een aantal vragen. Hier wordt voor elke subschaal gedefinieerd uit welke vragen deze bestaat, en welke subschalen er totaal zijn. 

In [12]:
# Definieer vragen per subschalen
doeltreffendheid = ['Vraag3', 'Vraag4', 'Vraag5', 'Vraag6']
affectiviteit = ['Vraag7', 'Vraag8', 'Vraag9']
cognitie = ['Vraag10', 'Vraag11', 'Vraag12', 'Vraag13']
collectief_eigenaarschap = ['Vraag14', 'Vraag15', 'Vraag16', 'Vraag17', 'Vraag18', 'Vraag19', 'Vraag20', 'Vraag21', 'Vraag22', 'Vraag23']
verbeteren_individueel = ['Vraag24','Vraag25','Vraag26','Vraag27','Vraag28']
verbeteren_collectief = ['Vraag29','Vraag30','Vraag31','Vraag32','Vraag33']

# Alle subschalen verzameld
subschalen = [doeltreffendheid, affectiviteit, cognitie, collectief_eigenaarschap, verbeteren_individueel, verbeteren_collectief]

In [24]:
alphas = []
for i in range(0,len(subschalen)):
    alpha = (pg.cronbach_alpha(data=qcidata[subschalen[i]], nan_policy='listwise'))[0]
    alphas.append(alpha)

alphas #Print alpha values


[0.7650986344675027,
 0.8043022019102172,
 0.7445429794217964,
 0.9293420660997821,
 0.8323448696014071,
 0.8472117882073255]

### Observatie: 
Cronbach's alpha waardes zijn soms afwijkend voor een schaal ten opzichte van de waardes berekend via de R functie `alpha`. Onduidelijk waarom. Eerst uitzoeken voordat onderstaande uitgevoerd kan worden!

### Subschaal-optimalisatie
Voor subschalen met een alpha lager dan de vooraf opgestelde threshold (alpha_threshold = 0.7) geldt dat de interne consistentie van deze subschaal niet voldoende is. Om de interne consistentie te verhogen is het mogelijk om te bekijken of het laten vallen van 1 vraag de alpha verbetert tot boven de threshold. 

In [ ]:
alpha_threshold = 0.7

In [38]:
def calculate_cronbach_alpha_with_omission(data):
    """
    Calculate Cronbach's alpha and alpha with each question omitted using the pingouin library.

    Parameters:
    - data: pandas DataFrame. Rows are observations, and columns are items.

    Returns:
    - result: pandas DataFrame. A DataFrame containing the following columns:
        - 'Question': Question numbers.
        - 'CronbachAlphaWithOmission': Cronbach's alpha with each question omitted.
        - 'CronbachAlpha': Overall Cronbach's alpha.

    Credits: ChatGPT
    """
    result_dict = {'Question': [], 'CronbachAlphaWithOmission': []}

    # Ensure that the data is numeric
    data = data.apply(pd.to_numeric, errors='coerce')

    # Calculate overall Cronbach's alpha
    overall_alpha = pg.cronbach_alpha(data)[0]

    # Iterate through each question and calculate alpha with omission
    for col in data.columns:
        omitted_data = data.drop(col, axis=1)
        alpha_with_omission = pg.cronbach_alpha(omitted_data)[0]

        result_dict['Question'].append(col)
        result_dict['CronbachAlphaWithOmission'].append(alpha_with_omission)

    # Create a DataFrame from the result dictionary
    result_df = pd.DataFrame(result_dict)
    result_df['CronbachAlpha'] = overall_alpha

    return result_df